In [ ]:

import os
import threading
import time
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from utils import crop_and_resize, cv2_to_pil

# Settings
YOLO_MODEL_NAME = "yolov8n.pt"  # ultralytics will auto-download small model if not present
COLOR_MODEL_PATH = "models/color_classifier.h5"  # produced by train script
IMG_SIZE = (160,160)

# Load models
print("Loading YOLO model (this may auto-download weights)...")
yolo = YOLO(YOLO_MODEL_NAME)
print("Loading color classifier...")
if not os.path.exists(COLOR_MODEL_PATH):
    print("Color classifier not found at", COLOR_MODEL_PATH)
    print("Run train_color_classifier.py first.")
    # still allow demo with dummy behavior
    color_model = None
else:
    color_model = load_model(COLOR_MODEL_PATH)

# If color_model is available, we need class mapping. We'll assume folder names 'blue' and 'other'
# If you used different folder names, adjust labels list accordingly.
labels = ['blue', 'other']

# GUI
class App:
    def __init__(self, root):
        self.root = root
        root.title("Car Colour & People Detector")

        self.panel = tk.Label(root)
        self.panel.pack()

        controls = tk.Frame(root)
        controls.pack(fill="x", padx=5, pady=5)

        tk.Button(controls, text="Open Image", command=self.open_image).pack(side="left")
        tk.Button(controls, text="Open Video", command=self.open_video).pack(side="left")
        tk.Button(controls, text="Open Webcam", command=self.open_webcam).pack(side="left")
        tk.Button(controls, text="Stop", command=self.stop).pack(side="left")

        self.info_label = tk.Label(root, text="Counts: ", font=("Arial", 12))
        self.info_label.pack()

        self.vs = None
        self.thread = None
        self.stopEvent = threading.Event()
        self.running = False

    def open_image(self):
        path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if not path:
            return
        self.start_source(path, is_video=False)

    def open_video(self):
        path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv")])
        if not path:
            return
        self.start_source(path, is_video=True)

    def open_webcam(self):
        self.start_source(0, is_video=True)

    def start_source(self, source, is_video=True):
        self.stop()
        self.stopEvent.clear()
        self.vs = cv2.VideoCapture(source) if is_video else cv2.imread(source)
        self.running = True
        self.thread = threading.Thread(target=self.video_loop, args=(source,is_video), daemon=True)
        self.thread.start()

    def stop(self):
        if self.running:
            self.stopEvent.set()
            time.sleep(0.1)
            if hasattr(self, "vs") and isinstance(self.vs, cv2.VideoCapture):
                try:
                    self.vs.release()
                except:
                    pass
            self.running = False

    def video_loop(self, source, is_video=True):
        try:
            while not self.stopEvent.is_set():
                if is_video:
                    ret, frame = self.vs.read()
                    if not ret:
                        break
                else:
                    # source is image path; read once
                    frame = cv2.imread(source)
                    # show single frame then break
                    self.process_and_show(frame)
                    break

                self.process_and_show(frame)

                # for webcam, small sleep to reduce CPU
                if source == 0:
                    time.sleep(0.02)
        except Exception as e:
            print("Exception in video loop:", e)
        finally:
            self.stop()

    def process_and_show(self, frame):
        orig = frame.copy()
        # Run YOLO detection for person(0) and car(2) (COCO indices)
        # classes param ensures we only detect person and car
        results = yolo.predict(frame, conf=0.35, iou=0.45, classes=[0,2], verbose=False)
        # results is a list; take first
        if len(results) == 0:
            res = None
        else:
            res = results[0]

        people_count = 0
        car_count = 0

        if res is not None and res.boxes is not None:
            boxes = res.boxes
            for box, cls in zip(boxes.xyxy, boxes.cls):
                cls = int(cls)
                x1, y1, x2, y2 = [int(v) for v in box]
                if cls == 0:  # person
                    people_count += 1
                    # draw person bbox in green
                    cv2.rectangle(orig, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(orig, "Person", (x1, y1-6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
                elif cls == 2:  # car
                    car_count += 1
                    # crop car and classify color
                    crop = crop_and_resize(frame, (x1,y1,x2,y2), size=IMG_SIZE)
                    is_blue = False
                    label_text = "Unknown"
                    if crop is not None and color_model is not None:
                        x = crop.astype("float32")/255.0
                        x = np.expand_dims(x, axis=0)
                        preds = color_model.predict(x)
                        idx = np.argmax(preds, axis=1)[0]
                        label_text = labels[idx]
                        is_blue = (label_text == 'blue')
                    else:
                        # fallback heuristic if no classifier: use average hue check (crude)
                        if crop is not None:
                            hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
                            h_mean = hsv[:,:,0].mean()
                            # approximate: hue around 100-140 is blue; this is rough
                            if 90 < h_mean < 140:
                                is_blue = True
                                label_text = 'blue(heuristic)'
                            else:
                                label_text = 'other(heuristic)'

                    # Draw rectangle: red for blue cars, blue for others
                    if is_blue:
                        color = (0,0,255)  # BGR red
                    else:
                        color = (255,0,0)  # BGR blue

                    cv2.rectangle(orig, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(orig, f"Car:{label_text}", (x1, y2+16), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # update info label
        info = f"People: {people_count}    Cars detected: {car_count}"
        self.info_label.config(text=info)

        # Show frame in Tkinter panel
        image = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(image)
        img_tk = ImageTk.PhotoImage(image=img_pil)
        self.panel.imgtk = img_tk
        self.panel.configure(image=img_tk)

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


Loading YOLO model (this may auto-download weights)...
Loading color classifier...


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 833ms/step
